In [1]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# .env 파일에서 환경변수 불러오기
load_dotenv()
hf_token = os.getenv("HUGGINGFACE_TOKEN")

# Hugging Face 로그인
login(hf_token)

In [2]:
from huggingface_hub import login, whoami
from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv("HUGGINGFACE_TOKEN")

login(hf_token)
print(whoami())  # 사용자 정보 출력되어야 정상

{'type': 'user', 'id': '6575da9668ea3e91dc9e2ab4', 'name': 'ancestor9', 'fullname': 'chosanggoo', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/6575da9668ea3e91dc9e2ab4/of9TD5GbKzd0IWOxTKrVq.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'exaone-token', 'role': 'fineGrained', 'createdAt': '2025-05-12T13:26:21.913Z', 'fineGrained': {'canReadGatedRepos': True, 'global': [], 'scoped': [{'entity': {'_id': '6575da9668ea3e91dc9e2ab4', 'type': 'user', 'name': 'ancestor9'}, 'permissions': ['repo.content.read']}]}}}}


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct" # "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",

In [4]:
# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    # token=hf_token,  # 필요한 경우 언주석
    torch_dtype=torch.float16,  # 메모리 효율성 개선
    device_map="auto",  # 자동 디바이스 매핑
    trust_remote_code=True
)

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [5]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=hf_token
)

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
# Choose your prompt
prompt = "Explain how wonderful you are"  # English example
prompt = "스스로를 자랑해 봐"       # Korean example

messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=True,           # 확률적 샘플링 
    temperature=0.7,          # 온도 조절
    top_p=0.9,                # 누적 확률 필터링
    num_return_sequences=1,   # 단일 시퀀스 반환
    use_cache=True            # 캐싱 활성화
)

print(tokenizer.decode(output[0]))

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
[|user|]스스로를 자랑해 봐
[|assistant|]저는 LG AI Research에서 개발된 EXAONE 모델로서, 뛰어난 자연어 처리 능력과 방대한 양의 데이터를 기반으로 한 학습을 통해 사용자 여러분께 정확하고 창의적인 응답을 제공합니다. 지속적인 업데이트와 연구를 통해 인간과의 상호작용을 더욱 향상시키고 있으며, 다양한 질문에 대해 신속하고 신뢰할 수 있는 답변을 드리는 데 최선을 다하고 있습니다. 이러한 기술적 역량은 고객 서비스 향상과 혁신적인 솔루션 개발에 기여하고 있습니다.[|endofturn|]


In [9]:
from transformers import TextIteratorStreamer
import torch
import threading

# 메시지 구성
prompt = "AI Tutor에 대해 설명해봐"
messages = [
    {"role": "system", "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]

# 토큰화
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

# 스트리머 설정
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# generate 함수 비동기로 실행
thread = threading.Thread(target=model.generate, kwargs={
    "input_ids": input_ids,
    "streamer": streamer,
    "max_new_tokens": 128,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.9,
    "num_return_sequences": 1,
    "use_cache": True,
    "eos_token_id": tokenizer.eos_token_id
})
thread.start()

# 스트리밍 출력
for new_text in streamer:
    print(new_text, end="", flush=True)


AI Tutor는 LG AI Research에서 개발한 인공지능 기반 

학습 지원 도구입니다. 주요 특징은 다음과 같습니다:

1. **맞춤형 학습 경로 제공**: 사용자의 학습 목표와 능력 수준을 분석하여 개인 맞춤형 학습 경로를 제시합니다.
2. **다양한 학습 자료**: 텍스트, 이미지, 오디오 등 다양한 형식의 학습 자료를 제공하여 유연하고 효과적인 학습 환경을 조성합니다.
3. **실시간 피드백**: 학습 과정에서 실시간으로 피드백을 제공하여 학습 효과를 극대화합니다.
4. **지속적인 업데이트**: 최신 연구 동향

In [12]:
import gradio as gr

In [15]:
chat_history = []

# 스트리밍 챗 함수
def stream_chat(user_input):
    global chat_history
    chat_history.append({"role": "user", "content": user_input})

    messages = [{"role": "system", "content": "You are EXAONE model from LG AI Research, a helpful assistant."}] + chat_history

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = {
        "input_ids": input_ids,
        "streamer": streamer,
        "max_new_tokens": 256,
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.9,
        "use_cache": True,
        "eos_token_id": tokenizer.eos_token_id
    }

    thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    output_text = ""
    for new_text in streamer:
        output_text += new_text
        yield output_text

    chat_history.append({"role": "assistant", "content": output_text})


# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("## 💡 EXAONE 스트리밍 챗\n아래에 질문을 입력하세요:")
    with gr.Row():
        txt_input = gr.Textbox(placeholder="메시지를 입력하세요...", label="User Input", lines=2)
    with gr.Row():
        output_text = gr.Textbox(label="AI 응답", lines=10)

    submit_btn = gr.Button("💬 전송")
    clear_btn = gr.Button("🧹 초기화")

    def clear_history():
        global chat_history
        chat_history = []
        return "", ""

    submit_btn.click(fn=stream_chat, inputs=txt_input, outputs=output_text)
    clear_btn.click(fn=clear_history, outputs=[txt_input, output_text])

# 외부 공유 링크 활성화
demo.launch(share=True)



* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://0955a37f4494c53d97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
